In [3]:
def slide_window(x, y, windowsize):
    datasize = x.shape[0] - windowsize + 1
    data = np.zeros([datasize, windowsize, x.shape[1]])
    label = np.zeros([datasize, windowsize, y.shape[1]])
    for i in range(0, datasize):
        temp = x[i:i+windowsize, :]
        data[i,:,:] = temp
        temp2 = y[i:i+windowsize, :]
        label[i,:,:] = temp2
    return data, label

In [2]:
# Predicted model training file

import numpy as np
import torch
import matplotlib.pyplot as plt
from PredictionModel import LSTM, device, LSTM2, LSTM3, ANN, GCN, normalize_sparse_adj
from tkinter import Variable
from Kinetics import cstr_cascading_kinetics
# parameters setting
Ea  = 50000     # activation energy J/gmol
R   = 8.314     # gas constant J/gmol/K
k0  = 8.46e6    # Arrhenius rate constant 1/min
V   = 1         # Volume [m3]
rho = 1000.0    # Density [kg/m3]
Cp  = 0.231     # Heat capacity [kJ/kg/K]
dHr = -1.15e4   # Enthalpy of reaction [kJ/kmol]
q   = 5.0       # Flowrate [m3/h]
cAi = 1.0       # Inlet feed concentration [mol/L]
Ti  = 350.0     # Inlet feed temperature [K]
cA0 = 0.5      # Initial concentration [mol/L]
T10 = 300    # Initial temperature of tank 1 [K]
T20 = 300     # Initial temperature of tank 2 [K]
kinetic1 = cstr_cascading_kinetics( Ea, R, k0, V, rho, Cp, dHr, q, Ti, cA0, T10, T20)

eps = 1e-50
trainRate = 0.7
valRate = 0.2
testRate = 0.
# generate data
#????????????????????
# data preprocess
windowsize = 5     

# read data
inputData = np.load("../data/windows/sontag/StateTraininputRecord_rand.npy")
outputData = np.load("../data/windows/sontag/StateTrainoutputRecord_state_rand.npy")
# inputData = np.load("data/windows/inputRecord.npy")
# outputData = np.load("data/windows/outputRecord_variation.npy")
# outputData_state = np.load("data/windows/outputRecord_state.npy")

# 归一化操作
#     X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
#
minInputValue = inputData.min(axis=1).min(axis=0)
maxInputValue = inputData.max(axis=1).max(axis=0)
minOutputValue = outputData.min(axis=1).min(axis=0)
maxOutputValue = outputData.max(axis=1).max(axis=0)

# 做归一化
inputData = (inputData - minInputValue) / (maxInputValue - minInputValue + eps)
outputData = (outputData - minOutputValue) / (maxOutputValue - minOutputValue + eps)

inputData = torch.Tensor(inputData).to(device)
outputData = torch.Tensor(outputData).to(device)

batchSize = 256
dataGroup = int(inputData.shape[0] / batchSize)
features = inputData.shape[2]
outputFeature = outputData.shape[2]
inputDataUse = torch.Tensor(np.zeros([dataGroup, batchSize, windowsize, features])).to(device)
outputDataUse = torch.Tensor(np.zeros([dataGroup, batchSize, outputFeature])).to(device)

for i in range(dataGroup):
    inputDataUse[i,:,:,:] = inputData[256*i:256*(i+1),:,:]
    outputDataUse[i,:,:] = outputData[256*i:256*(i+1),-1,:]

trainSizeNum = int(trainRate * dataGroup)
valSizeNum = int((trainRate+valRate) * dataGroup)

trainData = inputDataUse[0:trainSizeNum, :,:,:]
trainLable = outputDataUse[0:trainSizeNum, :,:]

valData = inputDataUse[trainSizeNum:valSizeNum, :,:,:]
valLabel = outputDataUse[trainSizeNum:valSizeNum, :,:]

testData = inputDataUse[valSizeNum:, :,:,:]
testLabel = outputDataUse[valSizeNum:, :,:]

## Train parameters
systemStateSize = features - 4
systemInput = 4
fullState = features + 1 # 多一个噪声项
hiddenSize = 64 # lstm 隐层个数
numlayer = 1    # lstm 层数
outputNum = outputFeature # 系统输出 - 状态量

num_epochs = 100
learning_rate = 0.001

In [17]:
from scipy import sparse
adj = np.array([[0,1,0,0,0,0,1,0,0,0],
                [1,0,0,0,0,0,0,1,0,0],
                [1,1,0,0,0,0,0,0,0,0],
                [1,0,0,0,1,0,0,0,1,0],
                [0,1,0,1,0,0,0,0,0,1],
                [0,0,1,1,1,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0,0]])
adj = sparse.csc_matrix(adj)
adj = normalize_sparse_adj(adj)

adj = torch.Tensor(adj.todense()).to(device)
print(inputDataUse.shape)

torch.Size([1000, 256, 5, 10])


In [27]:
print(adj.shape)
a = trainData[1,:,:,:]
a = a.view(a.shape[0], a.shape[-1], a.shape[1])

b = trainData[1,:,:,:]

c = torch.matmul(b,adj)
print(c.shape)
# d = torch.matmul(c, a)



torch.Size([10, 10])
torch.Size([256, 10, 5])
torch.Size([256, 5, 10])


In [2]:
import numpy as np
a = np.random.rand(3,3)
print(a)
print(a[-2:,-3])

[[0.49644762 0.34447364 0.5750177 ]
 [0.95102878 0.720702   0.95301651]
 [0.5365658  0.986983   0.12954661]]
[0.95102878 0.5365658 ]
